In [106]:
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
from sklearn.feature_selection import SelectKBest, r_regression
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.tree import DecisionTreeClassifier

SEED = 12345
VALID_RATIO = 0.2

OUTPUT_FILE = "data_v10_3.csv"

In [107]:
from google.colab import drive
drive.mount("/content/drive/")
INPUT_PATH_1 = "/content/drive/My Drive/HousePrices/input/"
INPUT_PATH_2 = "/content/drive/My Drive/HousePrices/features/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [108]:
df_train = pd.read_csv(INPUT_PATH_1 + "train.csv")
print(df_train.shape)
df_train.head(3)

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [109]:
df_train, df_valid = train_test_split(df_train, test_size=VALID_RATIO, shuffle=True, random_state=SEED)
print(df_train.shape, df_valid.shape)
df_valid.head(3)

(1168, 81) (292, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
986,987,50,RM,59.0,5310,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,117000
411,412,190,RL,100.0,34650,Pave,NaN,Reg,Bnk,AllPub,...,0,NaN,NaN,NaN,0,1,2006,WD,Normal,145000
1123,1124,20,RL,50.0,9405,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2009,WD,Normal,118000


In [110]:
df_train['train_test'] = 1
df_valid['train_test'] = 2
df_trainval = pd.concat([df_train, df_valid], axis=0).reset_index(drop=True)
print(df_trainval['train_test'].value_counts())
print(df_trainval.shape)
df_trainval.head(3)

1    1168
2     292
Name: train_test, dtype: int64
(1460, 82)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,train_test
0,784,85,RL,NaN,9101,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,7,2009,WD,Normal,165500,1
1,512,120,RL,40.0,6792,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,3,2006,New,Partial,202665,1
2,1072,60,RL,78.0,11700,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2009,WD,Normal,154000,1


In [111]:
df_test = pd.read_csv(INPUT_PATH_1 + "test.csv")
print(df_test.shape)
df_test.head(3)

(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal


In [112]:
df_test['SalePrice'] = np.nan
df_test['train_test'] = 3
print(df_test.shape)
df_test.head(3)

(1459, 82)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,train_test
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,0,6,2010,WD,Normal,NaN,3
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,Gar2,12500,6,2010,WD,Normal,NaN,3
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,NaN,MnPrv,NaN,0,3,2010,WD,Normal,NaN,3


In [113]:
df = pd.concat([df_trainval, df_test], axis=0).reset_index(drop=True)
#df_cash = df.copy()
print(df.shape)
df.head(3)

(2919, 82)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,train_test
0,784,85,RL,NaN,9101,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,7,2009,WD,Normal,165500.0,1
1,512,120,RL,40.0,6792,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,3,2006,New,Partial,202665.0,1
2,1072,60,RL,78.0,11700,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2009,WD,Normal,154000.0,1


In [114]:
columns = ['Id','GrLivArea', 'TotalBsmtSF', 'LotArea', 'BsmtFinSF1', 'GarageArea',
       'OverallQual', '1stFlrSF', 'YearBuilt', 'LotFrontage', 'Neighborhood']
columns = columns + ['YearRemodAdd', 'OpenPorchSF', 'OverallCond', 'GarageYrBlt', 'MoSold',
       '2ndFlrSF', 'BsmtUnfSF', 'WoodDeckSF', 'BsmtFinType1', 'MasVnrArea']
columns = columns + ['GarageCars', 'SaleCondition', 'GarageType', 'MSSubClass', 'Fireplaces',
       'LotShape', 'YrSold', 'MSZoning', 'BsmtFullBath', 'LandContour']
columns = columns + ['SalePrice','train_test']
df = df[columns]
print(df.shape)
df.head(3)

(2919, 33)


,Id,GrLivArea,TotalBsmtSF,LotArea,BsmtFinSF1,GarageArea,OverallQual,1stFlrSF,YearBuilt,LotFrontage,...,GarageType,MSSubClass,Fireplaces,LotShape,YrSold,MSZoning,BsmtFullBath,LandContour,SalePrice,train_test
0,784,1110,1097.0,9101,1097.0,602.0,5,1110,1978,NaN,...,Attchd,85,1,IR1,2009,RL,1.0,Lvl,165500.0,1
1,512,1368,1368.0,6792,0.0,474.0,7,1368,2005,40.0,...,Attchd,120,1,IR1,2006,RL,0.0,Lvl,202665.0,1
2,1072,1556,727.0,11700,298.0,441.0,6,829,1968,78.0,...,Attchd,60,0,Reg,2009,RL,0.0,Lvl,154000.0,1


In [115]:
# TOP10
column = 'TotalBsmtSF'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= df[column].median()
print(df[column].isnull().sum())

column = 'BsmtFinSF1'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= df[column].median()
print(df[column].isnull().sum())

column = 'GarageArea'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= df[column].median()
print(df[column].isnull().sum())

# TOP20
column = 'GarageYrBlt'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= df[column].median()
print(df[column].isnull().sum())

column = 'BsmtUnfSF'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= df[column].median()
print(df[column].isnull().sum())

column = 'BsmtFinType1'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= df[column].mode()[0]
print(df[column].isnull().sum())

column = 'MasVnrArea'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= df[column].median()
print(df[column].isnull().sum())

# TOP30
column = 'GarageCars'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= 0
print(df[column].isnull().sum())

column = 'GarageType'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= 'None'
print(df[column].isnull().sum())

column = 'MSZoning'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= 'None'
print(df[column].isnull().sum())

column = 'BsmtFullBath'
print(df[column].isnull().sum())
df.loc[df[column].isnull(), column]= df[column].mode()[0]
print(df[column].isnull().sum())

1
0
1
0
1
0
159
0
1
0
79
0
23
0
1
0
157
0
4
0
2
0


In [116]:
#df.isnull().sum()
#df['BsmtFullBath'].value_counts()

In [117]:
df['LotFrontage_null'] = 0
df.loc[df['LotFrontage'].isnull(), 'LotFrontage_null'] = 1
print(df.shape)

(2919, 34)


In [118]:
column = 'LotFrontage'
dict_tmp = df.groupby(['Neighborhood'])[column].median().to_dict()
df[column] = df.apply(lambda row: dict_tmp[row['Neighborhood']] if np.isnan(row[column]) else row[column], axis=1)

In [119]:
oe = OrdinalEncoder()
oe.set_output(transform='pandas')
cat_cols = ['Neighborhood','BsmtFinType1','SaleCondition','GarageType','LotShape','MSZoning','LandContour']
#cat_cols = ['Neighborhood','BsmtFinType1']
#cat_cols = df.select_dtypes(exclude=np.number).columns.to_list()
df[cat_cols] = oe.fit_transform(df[cat_cols])
print(cat_cols)
df.head(3)

['Neighborhood', 'BsmtFinType1', 'SaleCondition', 'GarageType', 'LotShape', 'MSZoning', 'LandContour']


,Id,GrLivArea,TotalBsmtSF,LotArea,BsmtFinSF1,GarageArea,OverallQual,1stFlrSF,YearBuilt,LotFrontage,...,MSSubClass,Fireplaces,LotShape,YrSold,MSZoning,BsmtFullBath,LandContour,SalePrice,train_test,LotFrontage_null
0,784,1110,1097.0,9101,1097.0,602.0,5,1110,1978,74.0,...,85,1,0.0,2009,4.0,1.0,3.0,165500.0,1,1
1,512,1368,1368.0,6792,0.0,474.0,7,1368,2005,40.0,...,120,1,0.0,2006,4.0,0.0,3.0,202665.0,1,0
2,1072,1556,727.0,11700,298.0,441.0,6,829,1968,78.0,...,60,0,3.0,2009,4.0,0.0,3.0,154000.0,1,0


In [120]:
# TOP10
df['Neighborhood'] = df['Neighborhood'].astype(np.int64)

#df['TotalBsmtSF'].value_counts()
#print(df['TotalBsmtSF'].min(), df['TotalBsmtSF'].max())
df['TotalBsmtSF'] = df['TotalBsmtSF'].astype(np.int64)

#df['BsmtFinSF1'].value_counts()
#print(df['BsmtFinSF1'].min(), df['BsmtFinSF1'].max())
df['BsmtFinSF1'] = df['BsmtFinSF1'].astype(np.int64)

#df['GarageArea'].value_counts()
#print(df['GarageArea'].min(), df['GarageArea'].max())
df['GarageArea'] = df['GarageArea'].astype(np.int64)

#df['LotFrontage'].value_counts()
#print(df['LotFrontage'].min(), df['LotFrontage'].max())
df['LotFrontage'] = df['LotFrontage'].astype(np.int64)

# TOP20
df['GarageYrBlt'] = df['GarageYrBlt'].astype(np.int64)
df['BsmtUnfSF'] = df['BsmtUnfSF'].astype(np.int64)
df['BsmtFinType1'] = df['BsmtFinType1'].astype(np.int64)
df['MasVnrArea'] = df['MasVnrArea'].astype(np.int64)
df['BsmtUnfSF'] = df['BsmtUnfSF'].astype(np.int64)

In [121]:
df['LandContour'].value_counts()

3.0    2622
1.0     120
0.0     117
2.0      60
Name: LandContour, dtype: int64

In [122]:
df.dtypes

Id                    int64
GrLivArea             int64
TotalBsmtSF           int64
LotArea               int64
BsmtFinSF1            int64
GarageArea            int64
OverallQual           int64
1stFlrSF              int64
YearBuilt             int64
LotFrontage           int64
Neighborhood          int64
YearRemodAdd          int64
OpenPorchSF           int64
OverallCond           int64
GarageYrBlt           int64
MoSold                int64
2ndFlrSF              int64
BsmtUnfSF             int64
WoodDeckSF            int64
BsmtFinType1          int64
MasVnrArea            int64
GarageCars          float64
SaleCondition       float64
GarageType          float64
MSSubClass            int64
Fireplaces            int64
LotShape            float64
YrSold                int64
MSZoning            float64
BsmtFullBath        float64
LandContour         float64
SalePrice           float64
train_test            int64
LotFrontage_null      int64
dtype: object

In [123]:
# 外れ値処理

In [124]:
df = df.drop(index=df[df['GrLivArea']==5642].index, axis=0)
df = df.drop(index=df[df['GrLivArea']==4676].index, axis=0)
print(df.shape)

(2917, 34)


In [125]:
# 丸め⇒△
# upper = df['SalePrice'].quantile(0.99)
# df.query('train_test!=3').loc[df.query('train_test!=3')['SalePrice']>upper, 'SalePrice'] = upper
# print(upper, df.query('train_test!=3')['SalePrice'].max())

In [126]:
# 削除⇒×
# upper = df['SalePrice'].quantile(0.99)
# df = df.drop(index=df.query('train_test!=3')[df.query('train_test!=3')['SalePrice']>upper].index, axis=0)
# print(df.shape)

In [127]:
# 特徴量エンジ

In [128]:
COL_CNT = 4
cat_cols = ['Neighborhood','OverallQual', 'YearBuilt','OverallCond','BsmtFinType1']
selector = SelectKBest(r_regression, k=COL_CNT)
selector.fit(df.query("train_test!=3").loc[:, cat_cols], df.query("train_test!=3")['SalePrice'])
cat_musk = selector.get_support()
#cat_musk = [True, True]
print(df[cat_cols].max())
print(cat_musk.sum())

Neighborhood      24
OverallQual       10
YearBuilt       2010
OverallCond        9
BsmtFinType1       5
dtype: int64
4


In [129]:
cat_new_list = []
com_list = list(itertools.combinations(np.array(cat_cols)[cat_musk], 2))
for i in range(len(com_list)):
  name = str(com_list[i][0]) + "_" + str(com_list[i][1])
  df[name] = df[com_list[i][0]]*10000 + df[com_list[i][1]]*1
  df[name] = df[name].astype(np.int16)
  cat_new_list.append(name)

print(cat_new_list)
print(df.shape)

['Neighborhood_OverallQual', 'Neighborhood_YearBuilt', 'Neighborhood_OverallCond', 'OverallQual_YearBuilt', 'OverallQual_OverallCond', 'YearBuilt_OverallCond']
(2917, 40)


In [130]:
NUM_CNT = 8
num_cols = df.drop(columns=['Id','SalePrice','train_test']+cat_cols+cat_new_list, axis=1).columns.tolist()
selector = SelectKBest(r_regression, k=NUM_CNT)
selector.fit(df.query("train_test!=3").loc[:, num_cols], df.query("train_test!=3")['SalePrice'])
num_musk = selector.get_support()
#num_cols = ['GrLivArea','TotalBsmtSF','LotArea']
#num_musk = [True, True, True]
print(num_musk.sum())

#cat_cols = ['Neighborhood','OverallQual']
#cat_musk = [True, True]

8


In [131]:
for num in np.array(num_cols)[num_musk]:
  for cat in np.array(cat_cols)[cat_musk]:
    name = str(num) + "_BY_" + str(cat)
    mean = df.loc[:, [cat, num]].groupby(cat, as_index=False).mean()
    mean = mean.rename(columns={num: 'tmp'})
    df = pd.merge(df, mean, on=cat)
    df[name] = df[num] - df['tmp']
    df[name] = df[name].astype(np.int64)
    df = df.drop(columns=['tmp'], axis=1)
    #display(df.tail(11))

print(df.shape)

(2917, 72)


In [132]:
#NUM_FEATURES = 40
#rfe = RFE(DecisionTreeClassifier(), n_features_to_select=NUM_FEATURES)
cv = KFold(n_splits=3, random_state=SEED, shuffle=True)
rfe = RFECV(DecisionTreeClassifier(random_state=SEED), cv=cv)
rfe.fit(df.query("train_test!=3").drop(columns=['Id','SalePrice','train_test'], axis=1), df.query("train_test!=3")['SalePrice'])
rfe.transform(df.query("train_test!=3").drop(columns=['Id','SalePrice','train_test'], axis=1))

array([[ 1110.,  1097.,  9101., ...,     0.,     0.,     0.],
       [  912.,   888.,  7540., ...,     0.,     0.,     0.],
       [ 1026.,   940., 10970., ...,     0.,     0.,     0.],
       ...,
       [ 1526.,  1013.,  8470., ...,     0.,     0.,     0.],
       [ 1856.,  1856., 11394., ...,     0.,     0.,     0.],
       [  904.,   683., 10020., ...,     0.,     0.,     0.]])

In [133]:
column = rfe.feature_names_in_[rfe.support_].tolist()
column = ['Id'] + ['OverallQual','Neighborhood','YearBuilt','OverallCond','BsmtFinType1','MoSold'] + column + ['SalePrice', 'train_test']
column = list(set(column))
df = df[column]
print(column)
print(df.shape)

['1stFlrSF_BY_Neighborhood', 'GarageCars', 'TotalBsmtSF_BY_OverallCond', 'MoSold', '1stFlrSF_BY_OverallCond', 'train_test', 'Neighborhood_OverallCond', 'Neighborhood_YearBuilt', 'GrLivArea_BY_OverallCond', 'Id', 'BsmtFinSF1', 'Neighborhood_OverallQual', 'YearRemodAdd_BY_Neighborhood', 'YearRemodAdd', 'YearRemodAdd_BY_YearBuilt', 'LotFrontage', '1stFlrSF', 'MasVnrArea_BY_OverallQual', 'MasVnrArea', 'GarageCars_BY_Neighborhood', 'OverallCond', 'Fireplaces_BY_Neighborhood', 'MasVnrArea_BY_YearBuilt', 'SaleCondition', '1stFlrSF_BY_YearBuilt', 'LotFrontage_null', 'YrSold', 'GarageType', 'GrLivArea_BY_YearBuilt', 'GarageCars_BY_OverallQual', 'TotalBsmtSF_BY_OverallQual', 'MSZoning', 'GarageCars_BY_YearBuilt', 'Fireplaces_BY_OverallCond', 'SalePrice', 'LandContour', 'BsmtFullBath', 'Fireplaces_BY_YearBuilt', 'TotalBsmtSF_BY_Neighborhood', '1stFlrSF_BY_OverallQual', 'GarageArea', 'OpenPorchSF', 'BsmtUnfSF', 'Fireplaces', 'WoodDeckSF', 'YearRemodAdd_BY_OverallQual', 'GarageArea_BY_YearBuilt', '

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2917 entries, 0 to 2916
Data columns (total 72 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   1stFlrSF_BY_Neighborhood      2917 non-null   int64  
 1   GarageCars                    2917 non-null   float64
 2   TotalBsmtSF_BY_OverallCond    2917 non-null   int64  
 3   MoSold                        2917 non-null   int64  
 4   1stFlrSF_BY_OverallCond       2917 non-null   int64  
 5   train_test                    2917 non-null   int64  
 6   Neighborhood_OverallCond      2917 non-null   int16  
 7   Neighborhood_YearBuilt        2917 non-null   int16  
 8   GrLivArea_BY_OverallCond      2917 non-null   int64  
 9   Id                            2917 non-null   int64  
 10  BsmtFinSF1                    2917 non-null   int64  
 11  Neighborhood_OverallQual      2917 non-null   int16  
 12  YearRemodAdd_BY_Neighborhood  2917 non-null   int64  
 13  Yea

In [135]:
df.to_csv(INPUT_PATH_2 + OUTPUT_FILE, index=False)
print(df.shape)
df.head(3)

(2917, 72)


,1stFlrSF_BY_Neighborhood,GarageCars,TotalBsmtSF_BY_OverallCond,MoSold,1stFlrSF_BY_OverallCond,train_test,Neighborhood_OverallCond,Neighborhood_YearBuilt,GrLivArea_BY_OverallCond,Id,...,LotArea,TotalBsmtSF_BY_YearBuilt,YearRemodAdd_BY_OverallCond,YearBuilt,GarageYrBlt,2ndFlrSF,LotShape,MasVnrArea_BY_OverallCond,BsmtFinType1,GarageArea_BY_OverallQual
0,-83,2.0,136,7,21,1,-21066,-19094,-247,784,...,9101,58,5,1978,1978,0,0.0,28,2,217
1,0,2.0,-96,5,85,3,-15530,-13558,-183,2171,...,12250,-174,5,1978,1978,0,0.0,104,0,143
2,-281,2.0,-72,6,-176,1,-21066,-19094,-445,361,...,7540,-150,5,1978,1978,0,0.0,-75,2,33
